In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
torch.cuda.set_device(0)
from fastai import *
from fastai.text import *
from pathlib import Path
import pickle

In [3]:
PATH=Path('/mnt/data/extracts/translate')
DATA=PATH/'data/'
TMP=PATH/'tmp/'
TMP.mkdir(exist_ok=True)
MODELS=Path('/mnt/models/translate')

## Load Data

In [4]:
qs = pickle.load((DATA/'fr-en-qs.pkl').open('rb'))
en_qs, fr_qs = zip(*qs)

In [5]:
en_tok = pickle.load((DATA/'en_tok.pkl').open('rb'))
fr_tok = pickle.load((DATA/'fr_tok.pkl').open('rb'))

In [7]:
en_tok[:10]

array([list(['what', 'is', 'light', '?']), list(['who', 'are', 'we', '?']),
       list(['where', 'did', 'we', 'come', 'from', '?']),
       list(['what', 'would', 'we', 'do', 'without', 'it', '?']),
       list(['what', 'is', 'the', 'absolute', 'location', '(', 'latitude', 'and', 'longitude', ')', 'of', 'badger', ',', 'newfoundland', 'and', 'labrador', '?']),
       list(['what', 'is', 'the', 'major', 'aboriginal', 'group', 'on', 'vancouver', 'island', '?']),
       list(['what', 'are', 'the', 'advantages', 'and', 'disadvantages', 'of', 'using', 'an', 'online', 'atlas', 'versus', 'a', 'paper', 'atlas', '?']),
       list(['what', 'types', 'of', 'land', 'cover', 'are', 'associated', 'with', 'the', 'colours', 'below', '?']),
       list(['what', 'is', 'the', 'population', 'of', 'canada', '?']),
       list(['which', 'province', 'is', 'the', 'most', 'populated', '?'])], dtype=object)

In [33]:
def tok2ids(tok, lim=40000):
    flat = [w for l in tok for w in l]
    toi = Counter(flat).most_common(lim)
    itos = [o for o, c in toi]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = {t:i for i, t in enumerate(itos)}
    return itos, stoi

In [34]:
abc, abc1 = tok2ids(fr_tok)

In [35]:
abc1

{'_bos_': 0,
 '_pad_': 1,
 '_eos_': 2,
 '_unk': 3,
 '?': 4,
 'de': 5,
 'les': 6,
 '-': 7,
 'la': 8,
 'le': 9,
 'des': 10,
 'et': 11,
 'à': 12,
 'est': 13,
 'sont': 14,
 'que': 15,
 ',': 16,
 'pour': 17,
 'quels': 18,
 'en': 19,
 'qui': 20,
 'du': 21,
 "l'": 22,
 'quelles': 23,
 'l’': 24,
 "d'": 25,
 't_up': 26,
 'dans': 27,
 'vous': 28,
 'pourquoi': 29,
 'quel': 30,
 'un': 31,
 'il': 32,
 'd’': 33,
 'une': 34,
 'quelle': 35,
 "qu'": 36,
 'sur': 37,
 '-ce': 38,
 'au': 39,
 'ou': 40,
 'a': 41,
 'qu’': 42,
 't': 43,
 'on': 44,
 'par': 45,
 'ils': 46,
 'ce': 47,
 'votre': 48,
 'plus': 49,
 'aux': 50,
 'pas': 51,
 'faire': 52,
 'si': 53,
 'canada': 54,
 'ces': 55,
 ')': 56,
 'ont': 57,
 'où': 58,
 'nous': 59,
 'être': 60,
 'se': 61,
 '-t': 62,
 '(': 63,
 'elle': 64,
 'été': 65,
 'cette': 66,
 'comment': 67,
 'quand': 68,
 'santé': 69,
 'peut': 70,
 'ne': 71,
 'autres': 72,
 'quoi': 73,
 'je': 74,
 'elles': 75,
 'programme': 76,
 '«': 77,
 '-il': 78,
 'fait': 79,
 'entre': 80,
 '»': 81,
 'y'

In [22]:
freq = Counter(abc)

In [23]:
len(freq)

17569

In [21]:
len(abc)

618582